In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-26 22:42:27.841865: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-26 22:42:28.069538: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-26 22:42:28.069560: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-26 22:42:28.070818: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-26 22:42:28.191216: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-26 22:42:30.993760: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 22:42:31.109011: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 22:42:31.119205: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:

"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.35,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.35,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2192 files for training.


2024-07-26 22:42:31.212655: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 22:42:31.215678: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-26 22:42:31.218535: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1180 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.efficientnet.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [12]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [13]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.EfficientNetB7(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [14]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: rescaling, False
Layer: normalization, False
Layer: rescaling_1, False
Layer: stem_conv_pad, False
Layer: stem_conv, False
Layer: stem_bn, False
Layer: stem_activation, False
Layer: block1a_dwconv, False
Layer: block1a_bn, False
Layer: block1a_activation, False
Layer: block1a_se_squeeze, False
Layer: block1a_se_reshape, False
Layer: block1a_se_reduce, False
Layer: block1a_se_expand, False
Layer: block1a_se_excite, False
Layer: block1a_project_conv, False
Layer: block1a_project_bn, False
Layer: block1b_dwconv, False
Layer: block1b_bn, False
Layer: block1b_activation, False
Layer: block1b_se_squeeze, False
Layer: block1b_se_reshape, False
Layer: block1b_se_reduce, False
Layer: block1b_se_expand, False
Layer: block1b_se_excite, False
Layer: block1b_project_conv, False
Layer: block1b_project_bn, False
Layer: block1b_drop, False
Layer: block1b_add, False
Layer: block1c_dwconv, False
Layer: block1c_bn, False
Layer: block1c_activation, False
Layer: block1c_se_sque

In [15]:
len(base_model.layers)

814

In [16]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [19]:
aug_layer = utils.return_data_aug_layer_for_eff_net(random_width=0.3, random_height=0.3, random_zoom=0.3, random_rotation=0.3)

In [23]:
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2

inputs = Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3), name='Input_layer')
x = aug_layer(inputs)
x = base_model(x, training=False)
x = GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(32, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

outputs = Dense(No_of_classes, activation='softmax', dtype=tf.float32)(x)

model_1 = Model(inputs, outputs, name='EfficientB7')

model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [24]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [25]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=2) 

In [ ]:
import pandas as pd
def save_history(history, name):
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(f'{name}/{name}.csv', index=False)

In [26]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [27]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 15, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/15


2024-07-26 20:37:57.734920: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-26 20:37:57.970260: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-26 20:37:59.435811: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x713e500034c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-26 20:37:59.435850: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-26 20:37:59.449912: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-26 20:37:59.547366: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


69/69 [==============================] - ETA: 0s - loss: 16.0270 - accuracy: 0.3800
Epoch 1: val_loss improved from inf to 13.50389, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 125s 2s/step - loss: 16.0270 - accuracy: 0.3800 - val_loss: 13.5039 - val_accuracy: 0.6186 - lr: 0.0010
Epoch 2/15
69/69 [==============================] - ETA: 0s - loss: 11.1185 - accuracy: 0.5406
Epoch 2: val_loss improved from 13.50389 to 9.16190, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 112s 2s/step - loss: 11.1185 - accuracy: 0.5406 - val_loss: 9.1619 - val_accuracy: 0.1153 - lr: 0.0010
Epoch 3/15
69/69 [==============================] - ETA: 0s - loss: 7.4122 - accuracy: 0.5949
Epoch 3: val_loss improved from 9.16190 to 6.34113, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 110s 2s/step - loss: 7.4122 - accuracy: 0.5949 - val_loss: 6.3411 - val_accuracy: 0.2220 - lr: 0.0010
Epoch 4/15
6

In [29]:
start_epoch = 15

In [31]:
history_2 = model_1.fit(train_datagen, validation_data = (val_datagen), initial_epoch= start_epoch, epochs = start_epoch+5 ,verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 16/20
69/69 [==============================] - ETA: 0s - loss: 1.0862 - accuracy: 0.7322
Epoch 16: val_loss improved from 1.27513 to 1.26643, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 106s 2s/step - loss: 1.0862 - accuracy: 0.7322 - val_loss: 1.2664 - val_accuracy: 0.5864 - lr: 3.0000e-04
Epoch 17/20
69/69 [==============================] - ETA: 0s - loss: 1.0267 - accuracy: 0.7427
Epoch 17: val_loss improved from 1.26643 to 1.09890, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 106s 2s/step - loss: 1.0267 - accuracy: 0.7427 - val_loss: 1.0989 - val_accuracy: 0.7119 - lr: 3.0000e-04
Epoch 18/20
69/69 [==============================] - ETA: 0s - loss: 0.9917 - accuracy: 0.7518
Epoch 18: val_loss did not improve from 1.09890
69/69 [==============================] - 32s 459ms/step - loss: 0.9917 - accuracy: 0.7518 - val_loss: 1.1143 - val_accuracy: 0.7102 - lr: 3.0000e-04
Epoch 19/20
69/69 [=============

In [34]:
start_epoch = 20

In [35]:
history_3 = model_1.fit(train_datagen, validation_data = (val_datagen), initial_epoch= start_epoch, epochs = start_epoch+15 ,verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/35
69/69 [==============================] - ETA: 0s - loss: 0.9876 - accuracy: 0.7477
Epoch 21: val_loss did not improve from 0.90306
69/69 [==============================] - 30s 433ms/step - loss: 0.9876 - accuracy: 0.7477 - val_loss: 1.1372 - val_accuracy: 0.6127 - lr: 3.0000e-04
Epoch 22/35
69/69 [==============================] - ETA: 0s - loss: 0.9820 - accuracy: 0.7464
Epoch 22: val_loss improved from 0.90306 to 0.86742, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 107s 2s/step - loss: 0.9820 - accuracy: 0.7464 - val_loss: 0.8674 - val_accuracy: 0.7983 - lr: 3.0000e-04
Epoch 23/35
69/69 [==============================] - ETA: 0s - loss: 0.9444 - accuracy: 0.7541
Epoch 23: val_loss did not improve from 0.86742
69/69 [==============================] - 31s 448ms/step - loss: 0.9444 - accuracy: 0.7541 - val_loss: 0.9320 - val_accuracy: 0.7483 - lr: 3.0000e-04
Epoch 24/35
69/69 [==============================] - ETA: 0s - loss: 0.9231 - 

In [36]:
base_model.trainable = True

In [37]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [38]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [39]:
start_epoch = 35
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [40]:
history_25 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 36/45
69/69 [==============================] - ETA: 0s - loss: 0.7974 - accuracy: 0.7924
Epoch 36: val_loss improved from 0.73500 to 0.72698, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 119s 2s/step - loss: 0.7974 - accuracy: 0.7924 - val_loss: 0.7270 - val_accuracy: 0.8212 - lr: 1.0000e-04
Epoch 37/45
69/69 [==============================] - ETA: 0s - loss: 0.7720 - accuracy: 0.8029
Epoch 37: val_loss did not improve from 0.72698
69/69 [==============================] - 27s 387ms/step - loss: 0.7720 - accuracy: 0.8029 - val_loss: 1.0841 - val_accuracy: 0.6025 - lr: 1.0000e-04
Epoch 38/45
69/69 [==============================] - ETA: 0s - loss: 0.7863 - accuracy: 0.8034
Epoch 38: val_loss improved from 0.72698 to 0.68241, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 99s 1s/step - loss: 0.7863 - accuracy: 0.8034 - val_loss: 0.6824 - val_accuracy: 0.8322 - lr: 1.0000e-04
Epoch 39/45
69/69 [==============

In [41]:
start_epoch = 45

In [42]:
history_45 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 46/55
69/69 [==============================] - ETA: 0s - loss: 0.6276 - accuracy: 0.8444
Epoch 46: val_loss improved from 0.53355 to 0.52745, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 103s 2s/step - loss: 0.6276 - accuracy: 0.8444 - val_loss: 0.5274 - val_accuracy: 0.8966 - lr: 3.0000e-05
Epoch 47/55
69/69 [==============================] - ETA: 0s - loss: 0.5986 - accuracy: 0.8590
Epoch 47: val_loss improved from 0.52745 to 0.50742, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 106s 2s/step - loss: 0.5986 - accuracy: 0.8590 - val_loss: 0.5074 - val_accuracy: 0.9008 - lr: 3.0000e-05
Epoch 48/55
69/69 [==============================] - ETA: 0s - loss: 0.5757 - accuracy: 0.8627
Epoch 48: val_loss improved from 0.50742 to 0.49251, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 108s 2s/step - loss: 0.5757 - accuracy: 0.8627 - val_loss: 0.4925 - val_accuracy: 0.9025 - l

In [43]:
start_epoch = 55
for layer in base_model.layers[-75:]:
    layer.trainable = True

In [44]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [45]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 56/65
69/69 [==============================] - ETA: 0s - loss: 1.2520 - accuracy: 0.6770
Epoch 56: val_loss did not improve from 0.44409
69/69 [==============================] - 47s 439ms/step - loss: 1.2520 - accuracy: 0.6770 - val_loss: 12.5162 - val_accuracy: 0.1559 - lr: 0.0010
Epoch 57/65
69/69 [==============================] - ETA: 0s - loss: 0.9120 - accuracy: 0.7400
Epoch 57: val_loss did not improve from 0.44409
69/69 [==============================] - 24s 348ms/step - loss: 0.9120 - accuracy: 0.7400 - val_loss: 5.5528 - val_accuracy: 0.0712 - lr: 0.0010
Epoch 58/65
69/69 [==============================] - ETA: 0s - loss: 0.8669 - accuracy: 0.7582
Epoch 58: val_loss did not improve from 0.44409
69/69 [==============================] - 25s 360ms/step - loss: 0.8669 - accuracy: 0.7582 - val_loss: 0.5624 - val_accuracy: 0.8881 - lr: 0.0010
Epoch 59/65
69/69 [==============================] - ETA: 0s - loss: 0.9088 - accuracy: 0.7322
Epoch 59: val_loss did not improve from 

In [46]:
start_epoch = 65

In [47]:
history_ = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 66/75
69/69 [==============================] - ETA: 0s - loss: 0.5598 - accuracy: 0.8435
Epoch 66: val_loss did not improve from 0.44409
69/69 [==============================] - 24s 342ms/step - loss: 0.5598 - accuracy: 0.8435 - val_loss: 0.4485 - val_accuracy: 0.9237 - lr: 2.7000e-05
Epoch 67/75
69/69 [==============================] - ETA: 0s - loss: 0.5608 - accuracy: 0.8572
Epoch 67: val_loss improved from 0.44409 to 0.42638, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 110s 2s/step - loss: 0.5608 - accuracy: 0.8572 - val_loss: 0.4264 - val_accuracy: 0.9246 - lr: 2.7000e-05
Epoch 68/75
69/69 [==============================] - ETA: 0s - loss: 0.5595 - accuracy: 0.8499
Epoch 68: val_loss did not improve from 0.42638
69/69 [==============================] - 24s 346ms/step - loss: 0.5595 - accuracy: 0.8499 - val_loss: 0.4686 - val_accuracy: 0.9144 - lr: 2.7000e-05
Epoch 69/75
69/69 [==============================] - ETA: 0s - loss: 0.5709 - 

In [48]:
start_epoch = 75
for layer in base_model.layers[-150:]:
    layer.trainable = True

In [49]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000006), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [50]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 76/85
69/69 [==============================] - ETA: 0s - loss: 0.5404 - accuracy: 0.8641
Epoch 76: val_loss did not improve from 0.39276
69/69 [==============================] - 47s 388ms/step - loss: 0.5404 - accuracy: 0.8641 - val_loss: 0.4107 - val_accuracy: 0.9195 - lr: 6.0000e-06
Epoch 77/85
69/69 [==============================] - ETA: 0s - loss: 0.5375 - accuracy: 0.8590
Epoch 77: val_loss did not improve from 0.39276
69/69 [==============================] - 22s 318ms/step - loss: 0.5375 - accuracy: 0.8590 - val_loss: 0.4017 - val_accuracy: 0.9178 - lr: 6.0000e-06
Epoch 78/85
69/69 [==============================] - ETA: 0s - loss: 0.5095 - accuracy: 0.8613
Epoch 78: val_loss improved from 0.39276 to 0.38639, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 121s 2s/step - loss: 0.5095 - accuracy: 0.8613 - val_loss: 0.3864 - val_accuracy: 0.9254 - lr: 6.0000e-06
Epoch 79/85
69/69 [==============================] - ETA: 0s - loss: 0.5262 - 

In [51]:
start_epoch = 85
for layer in base_model.layers[-250:]:
    layer.trainable = True

In [52]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0000008), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [53]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 86/95
69/69 [==============================] - ETA: 0s - loss: 0.5035 - accuracy: 0.8691
Epoch 86: val_loss improved from 0.38105 to 0.37959, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 147s 2s/step - loss: 0.5035 - accuracy: 0.8691 - val_loss: 0.3796 - val_accuracy: 0.9237 - lr: 8.0000e-07
Epoch 87/95
69/69 [==============================] - ETA: 0s - loss: 0.4935 - accuracy: 0.8682
Epoch 87: val_loss did not improve from 0.37959
69/69 [==============================] - 24s 336ms/step - loss: 0.4935 - accuracy: 0.8682 - val_loss: 0.3799 - val_accuracy: 0.9229 - lr: 8.0000e-07
Epoch 88/95
69/69 [==============================] - ETA: 0s - loss: 0.5223 - accuracy: 0.8577
Epoch 88: val_loss improved from 0.37959 to 0.37635, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 120s 2s/step - loss: 0.5223 - accuracy: 0.8577 - val_loss: 0.3764 - val_accuracy: 0.9237 - lr: 8.0000e-07
Epoch 89/95
69/69 [=============

In [54]:
start_epoch = 95
for layer in base_model.layers[-400:]:
    layer.trainable = True

In [55]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000000002), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [56]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 96/105
69/69 [==============================] - ETA: 0s - loss: 0.5063 - accuracy: 0.8704
Epoch 96: val_loss did not improve from 0.37635
69/69 [==============================] - 69s 500ms/step - loss: 0.5063 - accuracy: 0.8704 - val_loss: 0.3773 - val_accuracy: 0.9254 - lr: 2.0000e-09
Epoch 97/105
69/69 [==============================] - ETA: 0s - loss: 0.5178 - accuracy: 0.8672
Epoch 97: val_loss improved from 0.37635 to 0.37435, saving model to Trained_Models/EfficientB7
69/69 [==============================] - 127s 2s/step - loss: 0.5178 - accuracy: 0.8672 - val_loss: 0.3743 - val_accuracy: 0.9254 - lr: 2.0000e-09
Epoch 98/105
69/69 [==============================] - ETA: 0s - loss: 0.4866 - accuracy: 0.8768
Epoch 98: val_loss did not improve from 0.37435
69/69 [==============================] - 27s 384ms/step - loss: 0.4866 - accuracy: 0.8768 - val_loss: 0.3777 - val_accuracy: 0.9246 - lr: 2.0000e-09
Epoch 99/105
69/69 [==============================] - ETA: 0s - loss: 0.506

In [57]:
start_epoch = 105
for layer in base_model.layers[-600:]:
    layer.trainable = True

In [58]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000001), metrics = ['accuracy'])
model_1.summary()

Model: "EfficientB7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 efficientnetb7 (Functional  (None, 7, 7, 2560)        64097687  
 )                                                               
                                                                 
 global_average_pooling2d_4  (None, 2560)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 batch_normalization_5 (Bat  (None, 2560)              10240     
 chNormalization)                                      

In [ ]:
history_75 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+20, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 106/125


In [10]:
effb7_97 = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/EfficientB7")


In [11]:
effb7_97.evaluate(test_datagen)

2024-07-26 22:43:10.794602: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-26 22:43:10.998546: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


27/27 [==============================] - 8s 142ms/step - loss: 0.5178 - accuracy: 0.8450


[0.5178388357162476, 0.8449704051017761]